Preprocessing and Feature Extraction

In [ ]:
import numpy as np
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from sklearn.neighbors import NearestNeighbors
import os
import matplotlib.pyplot as plt
import cv2

# Load ResNet50 model without top layers
base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
model = Model(inputs=base_model.input, outputs=base_model.output)

# Function to preprocess and extract image features
def extract_features(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))
    img_data = image.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = preprocess_input(img_data)
    features = model.predict(img_data)
    return features

# Path to Dataset
dataset_path = './datasets'

# List all images and extract features
image_paths = []
features_list = []

for root, dirs, files in os.walk(dataset_path):
    for file in files:
        if file.endswith(('jpg', 'jpeg', 'png')):
            img_path = os.path.join(root, file)
            image_paths.append(img_path)
            features = extract_features(img_path, model)
            features_list.append(features.flatten())

# Convert list to numpy array
features_array = np.array(features_list)

# Simpan fitur dan path gambar
np.save('features.npy', features_array)
np.save('image_paths.npy', image_paths)


Indexing and Search Implementation

In [ ]:
# Load image features and paths
features_array = np.load('features.npy')
image_paths = np.load('image_paths.npy', allow_pickle=True)

# Create a Nearest Neighbors model
nn_model = NearestNeighbors(n_neighbors=5, metric='euclidean')
nn_model.fit(features_array)

# Function to search for similar images
def search_similar_images(query_img_path, model, nn_model, image_paths):
    query_features = extract_features(query_img_path, model).flatten().reshape(1, -1)
    distances, indices = nn_model.kneighbors(query_features)
    similar_images = [image_paths[idx] for idx in indices[0]]
    return similar_images

# Example of using search
query_image_path = 'test_images/image1366.jpeg'
similar_images = search_similar_images(query_image_path, model, nn_model, image_paths)

# Show search results
for img_path in similar_images:
    print(img_path)

def show_images(image_paths, query_img_path):
    plt.figure(figsize=(20, 10))
    
    # Display query image
    query_img = cv2.imread(query_img_path)
    query_img = cv2.cvtColor(query_img, cv2.COLOR_BGR2RGB)
    plt.subplot(2, 3, 1)
    plt.imshow(query_img)
    plt.title('Query Image')
    plt.axis('off')

    # Display search results images
    for i, img_path in enumerate(image_paths):
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.subplot(2, 3, i+2)
        plt.imshow(img)
        plt.title(f'Similar Image {i+1}')
        plt.axis('off')

    plt.show()

# Example of using search
query_image_path = 'test_images/image1366.jpeg'
similar_images = search_similar_images(query_image_path, model, nn_model, image_paths)

# Tampilkan hasil pencarian
show_images(similar_images, query_image_path)
